# 1) Set-up and standardisation
Loading morphometric data and demographic variables -> This will allow for further analysis of the dataset including contraction of morphometric similarity networks
 

In [ ]:
# Import key libraries:

import numpy as np                # Core array type and mathematical functions
import matplotlib
import matplotlib.pyplot as plt   # Plotting
from nilearn import plotting      # Brain network plotting
from scipy import stats           # Statistics
import bct as bct                 # Brain connectivity toolbox - network analysis
import random                     # Random number generator 
import pandas as pd               # Pandas
import statistics                 # To calculate mean

# Loading key demographic information from Maastricht dataset:

group_maas = np.loadtxt('group.dat',delimiter=',') # Group membership
age_maas = np.loadtxt('age.dat',delimiter=',')     # Age
sex_maas = np.loadtxt('sex.dat',delimiter=',')     # Sex

# Including  node coordinates for later analysis
coord_aal = np.loadtxt('centroids_500.txt', delimiter=',') # Node coordinates

# Inspecting data set to view demographics:
# Splitting patients into psychosis and controls
ind_healthy = np.where(group_maas==1)[0] # 1 represents healthy control
ind_patient = np.where(group_maas==2)[0] # 2 represents psychosis patient

# No. participants
ns_maas = len(group_maas) 
nhs_maas = len(ind_healthy)
nps_maas = len(ind_patient)

# Inspect data and print output:
print(f"The dataset consists of {ns_maas} subjects, including {sum(group_maas==1)} healthy controls and {sum(group_maas==2)} patients with psychosis.\n")
print(f"The subjects' ages range from {min(age_maas)} to {max(age_maas)} years.\n")
print(f"The dataset consists of {sum(sex_maas==1)} male and {sum(sex_maas==2)} female subjects.\n")

# Mean age
m_age = statistics.mean(age_maas)

print(f"The mean participant age was", m_age)

# 151 subjects
# 68 healthy controls
# 83 with psychosis
# Age range: 17-50
# 84 male, 67 female


# Morphometric features at each brain region for each participant:
morph_names = ['CT', 'FA', 'GC', 'GM', 'MC', 'MD', 'SA']  # Morphometric feature abbreviations
num_morph = len(morph_names)                             # Number of morphometric features

print(f"The dataset consists of {num_morph} morphometric features: {morph_names}. \n")

# Abbreviations correspond to the following:
# CT = cortical thickness
# FA = fractional anisotropy
# GC = gaussian curvature
# GM = grey matter volume
# MC = mean curvature
# MD = mean diffusivity
# SA = surface area


# Further key variables: 
# Number of regions and indices of upper triangular elements of the connectivity matrix
nnode_nspn = 308                             # Brain regions in atlas
triu_nspn = np.triu_indices(nnode_nspn, k=1) # Upper triangular indices

# Extracting .dat files of morphometric features to combine all of them accross features
morph_maas = np.zeros([ns_maas, nnode_nspn, num_morph])   # dimensions: subjects x regions x features

# Doing the same but split into patient groups (will need later)
morph_nhs = np.zeros([nhs_maas, nnode_nspn, num_morph])   # Dimensions: subjects x regions x features
morph_nps = np.zeros([nps_maas, nnode_nspn, num_morph])   # Dimensions: subjects x regions x features


# Loop over feature files to load data for BOTH GROUPS
for f in range(num_morph):
    
    # Load one feature file at a time
    temp_morph = np.genfromtxt('PARC500_'+morph_names[f]+'.dat',skip_header=0,skip_footer=0,dtype=None,delimiter='\n',encoding=None)
    
    # Loop over subjects to parse data
    for s in range(ns_maas):
        temp_morph_split = temp_morph[s].split(",")                       # split current feature at commas
        morph_maas[s,:,f] = np.array([float(i) for i in temp_morph_split]) # format into numpy array

####

# Loop over feature files to load data for HEALTHY CONTROLS
for f in range(num_morph):
    
    # Load one feature file at a time
    temp_morph_nhs = np.genfromtxt('PARC500_'+morph_names[f]+'.dat',skip_header=0,skip_footer=0,dtype=None,delimiter='\n',encoding=None)
    
    # Second loop over subjects to parse data
    for s in range(nhs_maas):
        temp_morph_nhs_split = temp_morph_nhs[s].split(",")                        # Split current feature at commas
        morph_nhs[s,:,f] = np.array([float(i) for i in temp_morph_nhs_split]) # Format into numpy array

####

# Loop over feature files to load data for PSYCHOSIS PATIENTS
for f in range(num_morph):
    
    # Load one feature file at a time
    temp_morph_nps = np.genfromtxt('PARC500_'+morph_names[f]+'.dat',skip_header=0,skip_footer=0,dtype=None,delimiter='\n',encoding=None)
    
    # Second loop over subjects to parse data
    for s in range(nps_maas):
        temp_morph_nps_split = temp_morph_nps[s].split(",")                        # Split current feature at commas
        morph_nps[s,:,f] = np.array([float(i) for i in temp_morph_nps_split]) # Format into numpy array  

# Clear temporary feature variables
del temp_morph
del temp_morph
del temp_morph_nhs
del temp_morph_nps
del temp_morph_nhs_split
del temp_morph_nps_split

# Intialise array to store standardised data- BOTH GROUPS
morph_maas_z = np.zeros(morph_maas.shape)   # we can use the shape of the existing feature array to set the shape of the new one

# Loop over features and subjects to standardise values across regions
for f in range(num_morph):
    for s in range(ns_maas):
        morph_maas_z[s,:,f] = stats.zscore(morph_maas[s,:,f])

# Finally, construct MSNs by correlating features between pairs of regions
# Intialise array to store MSNs
msn_maas = np.zeros([nnode_nspn, nnode_nspn, ns_maas])

# Compute pairwise correlation - between regions, across features - for each subject
for s in range(ns_maas):
    msn_maas[:,:,s] = np.corrcoef(morph_maas_z[s,:,:])
    
####


# Intialise array to store standardised data- HEALTHY CONTROLS
morph_nhs_z = np.zeros(morph_nhs.shape)   # we can use the shape of the existing feature array to set the shape of the new one

# Loop over features and subjects to standardise values across regions
for f in range(num_morph):
    for s in range(nhs_maas):
        morph_nhs_z[s,:,f] = stats.zscore(morph_nhs[s,:,f])

# Finally, construct MSNs by correlating features between pairs of regions
# Intialise array to store MSNs
msn_nhs = np.zeros([nnode_nspn, nnode_nspn, nhs_maas])

# Compute pairwise correlation - between regions, across features - for each subject
for s in range(nhs_maas):
    msn_nhs[:,:,s] = np.corrcoef(morph_nhs_z[s,:,:])
    
####

# Initialising array- this time for PSYCHOSIS PATIENTS
morph_nps_z = np.zeros(morph_nps.shape)   # we can use the shape of the existing feature array to set the shape of the new one

# Loop over features and subjects to standardise values across regions
for f in range(num_morph):
    for s in range(nps_maas):
        morph_nps_z[s,:,f] = stats.zscore(morph_nps[s,:,f])

# Finally, construct MSNs by correlating features between pairs of regions
# Intialise array to store MSNs
msn_nps = np.zeros([nnode_nspn, nnode_nspn, nps_maas])

# Compute pairwise correlation - between regions, across features - for each subject
for s in range(nps_maas):
    msn_nps[:,:,s] = np.corrcoef(morph_nps_z[s,:,:])

# Inspect the effect of data standardisation for a random healthy subject

# Set-up subplot grid
f, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Generate random number to pick random participant for standardisation example
num = random.randint(0,63)

# 1st subplot - un-normalised
ax = plt.subplot(1,2,1)                                               # Set-up subplot                     
plt.imshow(morph_nhs[num,:,:], interpolation='nearest', aspect='auto');# Plot morphometric features
plt.xlabel('Morphometric features', fontsize=15, labelpad=10);        # X-axis label
plt.ylabel('Regions', fontsize=15, labelpad=10);                      # Y-axis label	
plt.title('Un-normalised features', fontsize=15);                     # Title                      
ax.set_xticks(range(num_morph));                                      # Set axis range and tick labels
ax.set_xticklabels(morph_names);
plt.colorbar();   # Add colorbar


# 2nd subplot - normalised
ax = plt.subplot(1,2,2)
plt.imshow(morph_nhs_z[num,:,:], interpolation='nearest', aspect='auto');
plt.xlabel('Morphometric features', fontsize=15, labelpad=10);
plt.title('Normalised features', fontsize=15);
ax.set_xticks(range(num_morph));
ax.set_xticklabels(morph_names);

# Construct MSNs by correlating features between pairs of regions

# Intialise array to store MSNs
msn_maas = np.zeros([nnode_nspn, nnode_nspn, ns_maas])

# Compute pairwise correlation - between regions, across features - for each subject
for s in range(ns_maas):
    msn_maas[:,:,s] = np.corrcoef(morph_maas_z[s,:,:])

# Store first MSN as a separate "example" variable
msn_ex = msn_maas[:,:,0]

#####

# For HEALTHY CONTROLS:

# Intialise array to store MSNs
msn_nhs_maas = np.zeros([nnode_nspn, nnode_nspn, nhs_maas])

# Compute pairwise correlation - between regions, across features - for each subject
for s in range(nhs_maas):
    msn_maas[:,:,s] = np.corrcoef(morph_maas_z[s,:,:])

# Store first MSN as a separate "example" variable
msn_nhs_ex = msn_maas[:,:,0]

#####

# For PSYCHOSIS PATIENTS

# Intialise array to store MSNs
msn_nps_maas = np.zeros([nnode_nspn, nnode_nspn, nps_maas])

# Compute pairwise correlation - between regions, across features - for each subject
for s in range(nps_maas):
    msn_maas[:,:,s] = np.corrcoef(morph_maas_z[s,:,:])

# Store first MSN as a separate "example" variable
msn_nps_ex = msn_maas[:,:,0]


# Visualising MSN correlations

# HEALTHY CONTROLS

# Find indices for location of maximum (within the upper triangular part only)
ind_max_nhs = np.where(np.triu(msn_nhs_ex, k=1) == np.max(msn_nhs_ex[triu_nspn]))

# Extract normalised features for the two corresponding regions, for plotting
ex_max_zx_nhs = morph_nhs_z[0, ind_max_nhs[0][0], :]   # 1st region
ex_max_zy_nhs = morph_nhs_z[0, ind_max_nhs[1][0], :]   # 2nd region

# Plot maximum correlation
plt.figure(figsize=(4, 4))
plt.scatter(ex_max_zx_nhs, ex_max_zy_nhs)
plt.xlabel('Z-score', fontsize=15, labelpad=10)
plt.ylabel('Z-score', fontsize=15, labelpad=10)
plt.title('Maximum correlation in healthy controls')

# Annotate the scatterplot with feature names for each scatterplot marker
for i in range(num_morph):
plt.annotate(morph_names[i], (ex_max_zx_nhs[i], ex_max_zy_nhs[i] + 0.05))

####
    
# and PSYCHOSIS PATIENTS 
# Find indices for location of maximum (within the upper triangular part only)
ind_max_nps = np.where(np.triu(msn_nps_ex, k=1) == np.max(msn_nps_ex[triu_nspn]))

# Extract normalised features for the two corresponding regions, for plotting
ex_max_zx_nps = morph_nps_z[0, ind_max_nps[0][0], :]   # 1st region
ex_max_zy_nps = morph_nps_z[0, ind_max_nps[1][0], :]   # 2nd region

# Plot maximum correlation
plt.figure(figsize=(4, 4))
plt.scatter(ex_max_zx_nps, ex_max_zy_nps)
plt.xlabel('Z-score', fontsize=15, labelpad=10)
plt.ylabel('Z-score', fontsize=15, labelpad=10)
plt.title('Maximum correlation in psychosis patients')

# For the whole range, annotate the scatterplot with feature names for each scatterplot marker
for i in range(num_morph):
    plt.annotate(morph_names[i], (ex_max_zx_nps[i], ex_max_zy_nps[i] + 0.05))
    
# Verify the index of grey matter volume in the list of features
print(f'The first feature in the list is {morph_names[3]}.')

# Verify dimensions of grey matter volume data.
print(f'Dimensions of {morph_names[3]} data are {morph_maas_z[:,:,3].shape}.')

# Construct a structural covariance network
sc_nhs = np.corrcoef(morph_nhs_z[:,:,0].T) # we include the transpose, ".T", to correlate along the correct dimension.

# Visualise structural covariance network as a correlation matrix 
plt.imshow(sc_nhs, interpolation='nearest'); # plot the connectivity matrix
cbar = plt.colorbar();                          # Add colourbar
plt.title('Structural Covariance Matrix of Healthy Controls', fontsize=11); # Add title
cbar.set_label('<- High correlation low correlation ->', rotation=270) # Add colorbar label

# Adding threshold to healthy control strcutural covariance matrix to visualise output

# In this example, I am choosing a threshold of 45%

msn_thr = msn_nhs.copy()
threshold = 0.45                                                      # Set threshold at 0.45

for i in range(msn_thr.shape[0]):                                     # Iterate over rows
    for j in range(msn_thr.shape[1]):                                 # Iterate each value in row
        count = np.count_nonzero(msn_thr[i,j,:] > threshold, axis=0)  # Count num values above threshold across subjects
        if count/(msn_thr.shape[2]) < 0.75:                           # If proportion is < 0.75, set values to 0 
            msn_thr[i,j,:] = 0

plt.imshow(msn_thr[:,:,0], interpolation='nearest')                   # visualise the result - here, for the first participant
plt.colorbar();
plt.title('Thresholded Structural Covariance Matrix of Healthy Controls', fontsize=11); # Add title


# Verifying index of grey matter volume, but this time for psychosis patients

# Construct a structural covariance network
sc_nps = np.corrcoef(morph_nps_z[:,:,0].T) # we include the transpose, ".T", to correlate along the correct dimension.

# Visualise structural covariance network as a correlation matrix 
plt.imshow(sc_nps, interpolation='nearest'); # plot the connectivity matrix
cbar = plt.colorbar();                          # Add colourbar
plt.title('Structural Covariance Matrix of Psychosis Patients', fontsize=11); # Add title
cbar.set_label('<- High correlation low correlation ->', rotation=270) # Add colorbar label

# Adding threshold to patient strcutural covariance matrix to visualise output

msn_thr = msn_nps.copy()
threshold = 0.45                                                       # Set threshold at 0.45

for i in range(msn_thr.shape[0]):                                     # Iterate over rows
    for j in range(msn_thr.shape[1]):                                 # Iterate each value in row
        count = np.count_nonzero(msn_thr[i,j,:] > threshold, axis=0)  # Count num values above threshold across subjects
        if count/(msn_thr.shape[2]) < 0.75:                           # If proportion is < 0.75, set values to 0 
            msn_thr[i,j,:] = 0

plt.imshow(msn_thr[:,:,0], interpolation='nearest')                   # visualise the result - here, for the first participant
plt.colorbar();
plt.title('Thresholded Structural Covariance Matrix of Psychosis Patients', fontsize=11); # Add title



# An exploration of thresholding and visualisation of structural covariance and morphometric similarity networks.
# Use the coordinates for the NSPN_500 parcellation, below.

# Verify shape
print(f"The regional coordinates consist of {coord_aal.shape} values.\n")

# CONTROLS
# Example - structural covariance network
# Absolute thresholding 
threshold = 0.45
sc_thr = bct.threshold_absolute(sc_nhs, threshold)

# Visualize network with NSPN_500 coordinates
view = plotting.view_connectome(sc_thr, coord_aal)

#PSYCHOSIS
# Absolute thresholding 
threshold = 0.45
sc_thr = bct.threshold_absolute(sc_nps, threshold)

# Visualize network with NSPN_500 coordinates
view = plotting.view_connectome(sc_thr, coord_aal)


# 2) Proportional thresholding
Proportional thresholding is often preferred over absolute thresholding as it avoids systematic differences in absolute no. of edges
Here, the number of strongest connections are pre-defined (e.g. 0.1 retains only 10% of the strongest edges) and these are selected as network edges
 


 

In [ ]:
# Proportional thresholding of whole group- keeping x% of strongest connections
thr_prop = 0.2 # Change this number for different thresholds

# Guide:
# 0.1 = 10% of strongest connections
# 0.9 = 90%
# ...etc

msn_thr = msn_maas.copy()
for i in range(ns_maas):
    bct.threshold_proportional(msn_thr[:,:,i], thr_prop, copy = False)

# Visualise thresholded matrix (random subject selected)
plt.imshow(msn_thr[:,:,num], interpolation='nearest')      # visualise the result 
plt.colorbar();
plt.title('Proportional Thresholding of Psychosis and Healthy Controls', fontsize=11); # Add title

# Thresholding for healthy and psychosis groups:

# a) HEALTHY
thr_prop_nhs = 0.2 # Change this number for different thresholds

msn_thr_nhs = msn_nhs.copy()
for i in range(nhs_maas):
    bct.threshold_proportional(msn_thr_nhs[:,:,i], thr_prop_nhs, copy = False)

# b) PSYCHOSIS
thr_prop_nps = 0.2 # Change this number for different thresholds

msn_thr_nps = msn_nps.copy()
for i in range(nps_maas):
    bct.threshold_proportional(msn_thr_nps[:,:,i], thr_prop_nps, copy = False)




# 3) Global graph theoretical measurements
In this example, I have chosen to use characteristic path length

In [ ]:
# Calculate global graph theoretical measurement for each participant

# 1) CHARACTERISTIC PATH LENGTH

# HEALTHY 
# Start by setting up empty array to store measurement (length = number of participants)
msn_pl_nhs = np.zeros(msn_thr_nhs.shape[2])

# Calculate binary distance
#... Then path length 

for i in range(nhs_maas):	# Iterate through each subject and calculate chosen measurement

    msn_Dbin = bct.distance_bin(msn_thr[:,:,i])
    
    # It is important to set this to False if any additional statistical analyses are performed with the results
    msn_pl_nhs[i] = bct.charpath(msn_Dbin, include_infinite=False)[0] # Set to false in case extra statistical analysis is required
    
print(f'The binary characteristic path length of the structural connectome in healthy controls { round(msn_pl_nhs[i], 2) }.')

# PSYCHOSIS
# Start by setting up empty array to store measurement (length = number of participants)
msn_pl_nps = np.zeros(msn_thr_nps.shape[2])

# Same as above
for i in range(nps_maas):
msn_Dbin = bct.distance_bin(msn_thr[:,:,i])

    msn_pl_nps[i] = bct.charpath(msn_Dbin, include_infinite=False)[0] 

print(f'The binary characteristic path length of the structural connectome in psychosis patients is { round(msn_pl_nps[i], 2) }.')

# NODE DEGREE

#HEALTHY
nhs_bin = np.copy(ind_healthy) # Create a copy of the matrix 
nhs_bin[ind_healthy>0] = 1     # Binarise existing edges

node_deg_nhs = np.sum(nhs_bin, axis=0) # sum along the rows of the array 
print('the node degree in healthy participants is', node_deg_nhs)

# 67

# PSYCHOSIS
nps_bin = np.copy(ind_patient) # Create a copy of the matrix (NOTE: use np.copy() to copy numpy arrays)
nps_bin[ind_patient>0] = 1     # binarise existing edges

node_deg_nps = np.sum(nps_bin, axis=0) # sum along the rows of the array 
print('the node degree in psychosis participants is', node_deg_nps)

# 83

# NODE STRENGTH

# HEALTHY PARTICIPANTS
node_str_nhs = np.sum(ind_healthy, axis=0) # sum of weights along rows (again, columns would also work - with axis=1)

print('the node strength in healthy participants is', node_str_nhs)

# PSYCHOSIS PATIENTS
node_str_nps = np.sum(ind_patient, axis=0) # sum of weights along rows (again, columns would also work - with axis=1)

print('the node strength in psychosis participants is', node_str_nps)


# Compare healthy controls with psychosis patients using 'scipy.stats.ttest_ind'

# T TEST FOR PATH LENGTH:
# Calculate the t-test
t_stat_pl, p_val_pl = stats.ttest_ind(msn_pl_nhs, msn_pl_nps)

# Report results
print(f'The t-statistic for the difference in characteristic path length between healthy controls and patients with psychosis '
      f'based on MSNs thresholded to {100*thr_prop}% edge density, is t = {round(t_stat_pl,2)}. The corresponding p-value is p = {round(p_val_pl,2)}.')

# T TEST FOR PATH LENGTH:
# Calculate the t-test
t_stat_pl, p_val_pl = stats.ttest_ind(msn_pl_nhs, msn_pl_nps)

# Report results
print(f'The t-statistic for the difference in characteristic path length between healthy controls and patients with psychosis '
      f'based on MSNs thresholded to {100*thr_prop}% edge density, is t = {round(t_stat_pl,2)}. The corresponding p-value is p = {round(p_val_pl,2)}.')
